In [24]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv('train.csv')
df.info()
df.isna().sum()
df.duplicated().sum()
df.drop_duplicates()
label =  LabelEncoder()
df['model'] = label.fit_transform(df['model'])
df['motor_type'] = label.fit_transform(df['motor_type'])
df['wheel'] = label.fit_transform(df['wheel'])
df['color'] = label.fit_transform(df['color'])
df['status'] = label.fit_transform(df['status'])
df['type'] = label.fit_transform(df['type'])
def convert_running(value):
    if 'km' in value:
        kilometers = int(value.split()[0])  
        miles = kilometers * 0.621371        
        return f"{miles:.2f} miles"          
    elif 'miles' in value:
        return value  
    else:
        return value 
df['running'] = df['running'].apply(convert_running)
df['running'] = df['running'].str.replace('miles', '')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1642 entries, 0 to 1641
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         1642 non-null   object 
 1   year          1642 non-null   int64  
 2   motor_type    1642 non-null   object 
 3   running       1642 non-null   object 
 4   wheel         1642 non-null   object 
 5   color         1642 non-null   object 
 6   type          1642 non-null   object 
 7   status        1642 non-null   object 
 8   motor_volume  1642 non-null   float64
 9   price         1642 non-null   int64  
dtypes: float64(1), int64(2), object(7)
memory usage: 128.4+ KB


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1642 entries, 0 to 1641
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         1642 non-null   int32  
 1   year          1642 non-null   int64  
 2   motor_type    1642 non-null   int32  
 3   running       1642 non-null   object 
 4   wheel         1642 non-null   int32  
 5   color         1642 non-null   int32  
 6   type          1642 non-null   int32  
 7   status        1642 non-null   int32  
 8   motor_volume  1642 non-null   float64
 9   price         1642 non-null   int64  
dtypes: float64(1), int32(6), int64(2), object(1)
memory usage: 89.9+ KB


In [28]:
df['running'] = df['running'].str.strip()
df['running'] = pd.to_numeric(df['running'], errors='coerce')
df['running'] = df['running'].fillna(0).astype('int64')

In [30]:
df.corr()['price']

model           0.173485
year            0.638699
motor_type      0.238964
running        -0.513134
wheel                NaN
color          -0.036846
type            0.037789
status         -0.282471
motor_volume    0.000885
price           1.000000
Name: price, dtype: float64

In [38]:
x_train = df_no_outliers.drop(['price', 'wheel'], axis = 1)
y_train = df_no_outliers['price']
x_train.shape, y_train.shape

((1596, 8), (1596,))

In [32]:
Q1 = df['running'].quantile(0.25)
Q3 = df['running'].quantile(0.75)
IQR = Q3 - Q1

In [34]:
df_no_outliers = df[~((df['running'] < (Q1 - 1.5 * IQR)) | (df['running'] > (Q3 + 1.5 * IQR)))]

In [36]:
df_no_outliers

,model,year,motor_type,running,wheel,color,type,status,motor_volume,price
0,4,2022,3,1864,0,15,5,1,2.0,24500
1,2,2014,3,82020,0,1,5,1,2.0,25500
2,1,2018,3,95000,0,10,5,1,2.0,11700
3,2,2002,3,137000,0,6,5,1,3.2,12000
4,2,2017,3,80778,0,1,5,2,2.0,26000
...,...,...,...,...,...,...,...,...,...,...
1637,0,2017,3,120000,0,16,5,2,2.0,12400
1638,4,2014,3,105633,0,1,5,2,2.0,16500
1639,3,2018,3,68900,0,2,6,2,2.0,19500
1640,3,2019,3,31000,0,1,6,1,2.0,19500


In [40]:
from sklearn.preprocessing import StandardScaler
scalar =  StandardScaler()
feature_standardise = scalar.fit_transform(x_train)
feature_standardise

array([[ 1.50832144,  1.11734041,  0.25028438, ..., -0.0346266 ,
        -0.4393847 , -0.13285047],
       [ 0.05640285, -0.20409367,  0.25028438, ..., -0.0346266 ,
        -0.4393847 , -0.13285047],
       [-0.66955644,  0.45662337,  0.25028438, ..., -0.0346266 ,
        -0.4393847 , -0.13285047],
       ...,
       [ 0.78236214,  0.45662337,  0.25028438, ...,  1.07065458,
         1.03075781, -0.13285047],
       [ 0.78236214,  0.62180263,  0.25028438, ...,  1.07065458,
        -0.4393847 , -0.13285047],
       [ 1.50832144,  1.11734041,  0.25028438, ..., -0.0346266 ,
        -0.4393847 , -0.13285047]])

In [42]:
from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor(n_neighbors=7)  

knn.fit(feature_standardise, y_train)

KNeighborsRegressor(n_neighbors=7)

In [44]:
from sklearn.model_selection import GridSearchCV


param_grid = {'n_neighbors': [1, 3, 5, 7, 9, 11]}


grid_search = GridSearchCV(KNeighborsRegressor(), param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(feature_standardise, y_train)

print(f'Best number of neighbors: {grid_search.best_params_["n_neighbors"]}')

Best number of neighbors: 7


In [46]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

y_pred = knn.predict(feature_standardise)
mse = mean_squared_error(y_train, y_pred)
print(f'Mean Squared Error: {mse:.2f}')
mse = mean_absolute_error(y_train, y_pred)
print(f'Mean Absolute Error: {mse:.2f}')
r2 = r2_score(y_train, y_pred)
print(f"r2 Score = {r2}")

Mean Squared Error: 15272802.39
Mean Absolute Error: 2233.00
r2 Score = 0.6955023857556353


In [48]:
test = pd.read_csv('test.csv')
test

,Id,model,year,motor_type,running,wheel,color,type,status,motor_volume
0,0,kia,2020,petrol,24000 km,left,black,sedan,excellent,2.0
1,1,nissan,2017,petrol,85000 miles,left,white,suv,excellent,2.0
2,2,hyundai,2021,petrol,30000 miles,left,white,sedan,excellent,2.0
3,3,kia,2018,petrol,53000 miles,left,silver,sedan,excellent,2.0
4,4,mercedes-benz,2003,petrol,230000 km,left,black,Universal,normal,1.8
...,...,...,...,...,...,...,...,...,...,...
406,406,nissan,2021,petrol,33400 km,left,black,suv,excellent,2.0
407,407,hyundai,2017,petrol,60000 miles,left,other,sedan,excellent,2.0
408,408,mercedes-benz,2012,petrol,218000 km,left,white,sedan,good,2.0
409,409,kia,2020,petrol,40000 miles,left,red,sedan,good,2.0


In [50]:
from sklearn.preprocessing import LabelEncoder
label =  LabelEncoder()
test['model'] = label.fit_transform(test['model'])
test['motor_type'] = label.fit_transform(test['motor_type'])
test['wheel'] = label.fit_transform(test['wheel'])
test['color'] = label.fit_transform(test['color'])
test['status'] = label.fit_transform(test['status'])
test['type'] = label.fit_transform(test['type'])
def convert_runnings(value):
    if 'km' in value:
        kilometers = int(value.split()[0])  
        miles = kilometers * 0.621371        
        return f"{miles:.2f} miles"          
    elif 'miles' in value:
        return value  
    else:
        return value 
test['running'] = test['running'].apply(convert_runnings)
test['running'] = test['running'].str.replace('miles', '')

In [52]:
test.head()

,Id,model,year,motor_type,running,wheel,color,type,status,motor_volume
0,0,1,2020,1,14912.90,0,1,3,1,2.0
1,1,3,2017,1,85000,0,13,4,1,2.0
2,2,0,2021,1,30000,0,13,3,1,2.0
3,3,1,2018,1,53000,0,11,3,1,2.0
4,4,2,2003,1,142915.33,0,1,1,4,1.8


In [54]:
test

,Id,model,year,motor_type,running,wheel,color,type,status,motor_volume
0,0,1,2020,1,14912.90,0,1,3,1,2.0
1,1,3,2017,1,85000,0,13,4,1,2.0
2,2,0,2021,1,30000,0,13,3,1,2.0
3,3,1,2018,1,53000,0,11,3,1,2.0
4,4,2,2003,1,142915.33,0,1,1,4,1.8
...,...,...,...,...,...,...,...,...,...,...
406,406,3,2021,1,20753.79,0,1,4,1,2.0
407,407,0,2017,1,60000,0,9,3,1,2.0
408,408,2,2012,1,135458.88,0,13,3,2,2.0
409,409,1,2020,1,40000,0,10,3,2,2.0


In [105]:
x_test = test

In [107]:
x_test = x_test.drop(['Id', 'wheel'], axis = 1)

In [109]:
x_test

,model,year,motor_type,running,color,type,status,motor_volume
0,1,2020,1,14912.90,1,3,1,2.0
1,3,2017,1,85000,13,4,1,2.0
2,0,2021,1,30000,13,3,1,2.0
3,1,2018,1,53000,11,3,1,2.0
4,2,2003,1,142915.33,1,1,4,1.8
...,...,...,...,...,...,...,...,...
406,3,2021,1,20753.79,1,4,1,2.0
407,0,2017,1,60000,9,3,1,2.0
408,2,2012,1,135458.88,13,3,2,2.0
409,1,2020,1,40000,10,3,2,2.0


In [111]:
x_test['running'] = x_test['running'].str.strip()
x_test['running'] = pd.to_numeric(x_test['running'], errors='coerce')
x_test['running'] = x_test['running'].fillna(0).astype('int64')

In [64]:
Q1 = x_test['running'].quantile(0.25)
Q3 = x_test['running'].quantile(0.75)
IQR = Q3 - Q1

In [66]:
df_no_outliers_t = x_test[~((x_test['running'] < (Q1 - 1.5 * IQR)) | (x_test['running'] > (Q3 + 1.5 * IQR)))]
df_no_outliers_t

,model,year,motor_type,running,color,type,status,motor_volume
0,1,2020,1,14912,1,3,1,2.0
1,3,2017,1,85000,13,4,1,2.0
2,0,2021,1,30000,13,3,1,2.0
3,1,2018,1,53000,11,3,1,2.0
4,2,2003,1,142915,1,1,4,1.8
...,...,...,...,...,...,...,...,...
406,3,2021,1,20753,1,4,1,2.0
407,0,2017,1,60000,9,3,1,2.0
408,2,2012,1,135458,13,3,2,2.0
409,1,2020,1,40000,10,3,2,2.0


In [68]:
x_test = df_no_outliers_t

In [113]:
from sklearn.preprocessing import StandardScaler
scalar =  StandardScaler()
feature_standard = scalar.fit_transform(x_test)
feature_standard

array([[-0.62247891,  0.77117487,  0.1137996 , ..., -0.05892716,
        -0.4470452 , -0.10974538],
       [ 0.8692927 ,  0.31182288,  0.1137996 , ...,  1.45476435,
        -0.4470452 , -0.10974538],
       [-1.36836471,  0.9242922 ,  0.1137996 , ..., -0.05892716,
        -0.4470452 , -0.10974538],
       ...,
       [ 0.1234069 , -0.45376377,  0.1137996 , ..., -0.05892716,
         1.04673998, -0.10974538],
       [-0.62247891,  0.77117487,  0.1137996 , ..., -0.05892716,
         1.04673998, -0.10974538],
       [-1.36836471,  0.31182288,  0.1137996 , ..., -0.05892716,
        -0.4470452 , -0.10974538]])

In [115]:
y_predict = knn.predict(feature_standard)
y_predict
df_train = pd.DataFrame(y_predict, columns=['price'])
df_train

,price
0,16171.428571
1,16357.142857
2,24364.285714
3,16785.714286
4,8592.857143
...,...
406,18900.000000
407,14228.571429
408,14528.571429
409,14642.857143


In [74]:
combined_dfs = pd.concat([test['Id'], df_train], axis=1)

combined_dfs

,Id,price
0,0,17314.285714
1,1,16428.571429
2,2,24578.571429
3,3,16585.714286
4,4,7000.000000
...,...,...
406,406,NaN
407,407,NaN
408,408,NaN
409,409,NaN


In [122]:
combined_dfs.to_csv('F_sub.csv', index=False)

In [124]:
su = pd.read_csv('F_sub.csv')
su

,Id,price
0,0,17314.285714
1,1,16428.571429
2,2,24578.571429
3,3,16585.714286
4,4,7000.000000
...,...,...
406,406,NaN
407,407,NaN
408,408,NaN
409,409,NaN


In [179]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
reg = RandomForestRegressor(n_estimators=100)
model = RandomForestRegressor()
model.fit(feature_standardise,y_train)

RandomForestRegressor()

In [181]:
mse_R = mean_squared_error(y_train,  model.predict(feature_standardise))
print(f"Mean Squared Error = {mse_R}")

mae_R = mean_squared_error(y_train,  model.predict(feature_standardise))
print(f"Mean Absolute Error = {mae_R}")

Mean Squared Error = 2069352.978779534
Mean Absolute Error = 2069352.978779534


In [185]:
r2_R = r2_score(y_train,  model.predict(feature_standardise))
print(f"r2 Score = {r2_R}")

r2 Score = 0.9587428011713757


In [189]:
y_pred = model.predict(feature_standard)
y_pred
df_tra = pd.DataFrame(y_pred, columns=['price'])
df_tra

,price
0,17474.000000
1,16168.714286
2,23300.000000
3,15006.000000
4,7454.500000
...,...
406,29079.990000
407,14086.500000
408,14854.500000
409,17052.790000


In [191]:
comb_df = pd.concat([test['Id'], df_tra], axis=1)

comb_df

,Id,price
0,0,17474.000000
1,1,16168.714286
2,2,23300.000000
3,3,15006.000000
4,4,7454.500000
...,...,...
406,406,29079.990000
407,407,14086.500000
408,408,14854.500000
409,409,17052.790000


In [193]:
comb_df.to_csv('F_su.csv', index=False)
su = pd.read_csv('F_su.csv')
su

,Id,price
0,0,17474.000000
1,1,16168.714286
2,2,23300.000000
3,3,15006.000000
4,4,7454.500000
...,...,...
406,406,29079.990000
407,407,14086.500000
408,408,14854.500000
409,409,17052.790000


In [117]:
from xgboost import XGBRegressor

In [119]:
model_x = XGBRegressor(objective='reg:squarederror')
model_x.fit(feature_standardise, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [121]:
mse_X = mean_squared_error(y_train,  model_x.predict(feature_standardise))
print(f"Mean Squared Error = {mse_X}")

Mean Squared Error = 1098889.97607611


In [123]:
r2_X = r2_score(y_train,  model_x.predict(feature_standardise))
print(f"r2 Score = {r2_X}")

r2 Score = 0.9780911803245544


In [125]:
y_pred = model_x.predict(feature_standard)
y_pred
df_tra = pd.DataFrame(y_pred, columns=['price'])
df_tra

,price
0,14708.585938
1,11624.106445
2,13733.666016
3,11310.692383
4,7361.780273
...,...
406,14251.917969
407,11052.129883
408,13553.656250
409,13444.468750


In [127]:
com = pd.concat([test['Id'], df_tra], axis=1)

com

,Id,price
0,0,14708.585938
1,1,11624.106445
2,2,13733.666016
3,3,11310.692383
4,4,7361.780273
...,...,...
406,406,14251.917969
407,407,11052.129883
408,408,13553.656250
409,409,13444.468750


In [129]:
com.to_csv('Final.csv', index=False)
su = pd.read_csv('Final.csv')
su

,Id,price
0,0,14708.5860
1,1,11624.1060
2,2,13733.6660
3,3,11310.6920
4,4,7361.7803
...,...,...
406,406,14251.9180
407,407,11052.1300
408,408,13553.6560
409,409,13444.4690


,Id,price
0,0,14708.5860
1,1,11624.1060
2,2,13733.6660
3,3,11310.6920
4,4,7361.7803
...,...,...
406,406,14251.9180
407,407,11052.1300
408,408,13553.6560
409,409,13444.4690


In [4]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor

In [6]:
# Define the model
ran_model = RandomForestRegressor()

# Define the hyperparameter space
param_dist = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}

In [10]:
random_search = RandomizedSearchCV(
    estimator=ran_model,
    param_distributions=param_dist,
    n_iter=10,  # Number of random combinations to try
    cv=5,       # Number of cross-validation folds
    verbose=2,
    random_state=42,
    n_jobs=-1   # Use all available cores
)

# Fit the model
random_search.fit(feature_standardise, y_train)

NameError: name 'feature_standardise' is not defined

In [234]:
# Best parameters found
print("Best Parameters: ", random_search.best_params_)

# Make predictions
y_pred = random_search.predict(feature_standard)

# Calculate Mean Squared Error
# mse = mean_squared_error(y_test, y_pred)
# print("Mean Squared Error: ", mse)

Best Parameters:  {'n_estimators': 50, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_depth': 10}


In [242]:
ran_model = RandomForestRegressor(n_estimators= 50, min_samples_split= 5, min_samples_leaf= 1, max_depth=10)
ran_model.fit(feature_standardise, y_train)

RandomForestRegressor(max_depth=10, min_samples_split=5, n_estimators=50)

In [252]:
mse_Re = mean_squared_error(y_train,  random_search.predict(feature_standardise))
print(f"Mean Squared Error = {mse_Re}")

Mean Squared Error = 4814299.31715455


In [254]:
mse_Re = mean_absolute_error(y_train,  random_search.predict(feature_standardise))
print(f"Mean Squared Error = {mse_Re}")

Mean Squared Error = 1380.1054126511776


In [258]:
mse_Res = r2_score(y_train,  random_search.predict(feature_standardise))
print(f"Mean Squared Error = {mse_Res}")

Mean Squared Error = 0.9040161315226652


In [260]:
y= random_search.predict(feature_standard)
y
d = pd.DataFrame(y, columns=['price'])
d

,price
0,17221.664796
1,16432.824079
2,23240.766289
3,14548.054823
4,7654.474315
...,...
406,27150.195619
407,13950.407225
408,14746.594135
409,16548.542675


In [262]:
c = pd.concat([test['Id'], df_tra], axis=1)

c

,Id,price
0,0,17474.000000
1,1,16168.714286
2,2,23300.000000
3,3,15006.000000
4,4,7454.500000
...,...,...
406,406,29079.990000
407,407,14086.500000
408,408,14854.500000
409,409,17052.790000


In [264]:
c.to_csv('F.csv', index=False)
s = pd.read_csv('F.csv')
s

,Id,price
0,0,17474.000000
1,1,16168.714286
2,2,23300.000000
3,3,15006.000000
4,4,7454.500000
...,...,...
406,406,29079.990000
407,407,14086.500000
408,408,14854.500000
409,409,17052.790000


In [133]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Define the model
model = RandomForestRegressor()

# Set the hyperparameters you want to tune
param_grid = {
    'n_estimators': [50, 100, 200],      # Number of trees in the forest
    'max_features': ['auto', 'sqrt'],    # Number of features to consider at every split
    'max_depth': [None, 10, 20, 30],     # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],     # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],       # Minimum number of samples required to be at a leaf node
    'bootstrap': [True, False]            # Whether to use bootstrap samples
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, 
                           cv=3, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')

# Fit the model
grid_search.fit(feature_standardise, y_train)



Fitting 3 folds for each of 432 candidates, totalling 1296 fits


C:\Users\admin\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
648 fits failed out of a total of 1296.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
391 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\admin\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\admin\anaconda3\Lib\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "C:\Users\admin\anaconda3\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "C:\Users\admin\anaconda3\Lib\site-packa

GridSearchCV(cv=3, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'bootstrap': [True, False],
                         'max_depth': [None, 10, 20, 30],
                         'max_features': ['auto', 'sqrt'],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 200]},
             scoring='neg_mean_squared_error', verbose=2)

In [137]:
# Best parameters found
print("Best Parameters:", grid_search.best_params_)

# Use the best estimator to predict
best_model = grid_search.best_estimator_
y_pred = best_model.predict(feature_standardise)

# Evaluate the model
mae = mean_absolute_error(y_train, y_pred)
mse = mean_squared_error(y_train, y_pred)
r2 = r2_score(y_train, y_pred)

print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

Best Parameters: {'bootstrap': False, 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 50}
Mean Absolute Error: 1252.1935429546088
Mean Squared Error: 4218033.646239567
R^2 Score: 0.9159040267207726


In [139]:
y1= grid_search.predict(feature_standard)
y1
d1 = pd.DataFrame(y1, columns=['price'])
d1

,price
0,17242.214286
1,16421.430556
2,22701.085714
3,14920.842857
4,8474.528968
...,...
406,28100.500714
407,13979.851984
408,13383.383095
409,15629.934857


In [141]:
c1 = pd.concat([test['Id'], d1], axis=1)

c1


,Id,price
0,0,17242.214286
1,1,16421.430556
2,2,22701.085714
3,3,14920.842857
4,4,8474.528968
...,...,...
406,406,28100.500714
407,407,13979.851984
408,408,13383.383095
409,409,15629.934857


In [143]:
c1.to_csv('F1.csv', index=False)
s1 = pd.read_csv('F1.csv')
s1

,Id,price
0,0,17242.214286
1,1,16421.430556
2,2,22701.085714
3,3,14920.842857
4,4,8474.528968
...,...,...
406,406,28100.500714
407,407,13979.851984
408,408,13383.383095
409,409,15629.934857


In [145]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

# Define the model
model = RandomForestRegressor()

# Set the hyperparameters you want to tune
param_distributions = {
    'n_estimators': [50, 100, 200],
    'max_features': ['auto', 'sqrt'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_distributions, 
                                   n_iter=100, cv=3, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error', random_state=42)

# Fit the model
random_search.fit(feature_standardise, y_train)

# Best parameters found
print("Best Parameters:", random_search.best_params_)

# Use the best estimator to predict
best_model = random_search.best_estimator_
y_pred = best_model.predict(feature_standardise)

# Evaluate the model
mae = mean_absolute_error(y_train, y_pred)
mse = mean_squared_error(y_train, y_pred)
r2 = r2_score(y_train, y_pred)

print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

Fitting 3 folds for each of 100 candidates, totalling 300 fits


C:\Users\admin\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
171 fits failed out of a total of 300.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
134 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\admin\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\admin\anaconda3\Lib\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "C:\Users\admin\anaconda3\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "C:\Users\admin\anaconda3\Lib\site-packag

Best Parameters: {'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 20, 'bootstrap': False}
Mean Absolute Error: 829.0487271167351
Mean Squared Error: 1885460.3396488903
R^2 Score: 0.9624091139046476


In [147]:

y2= random_search.predict(feature_standard)
y2
d2 = pd.DataFrame(y2, columns=['price'])
d2

,price
0,16961.997380
1,16644.733417
2,23029.462500
3,14841.529000
4,8503.470833
...,...
406,28386.700833
407,13802.877727
408,14093.033333
409,15549.595417


In [149]:
c2 = pd.concat([test['Id'], d2], axis=1)

c2

,Id,price
0,0,16961.997380
1,1,16644.733417
2,2,23029.462500
3,3,14841.529000
4,4,8503.470833
...,...,...
406,406,28386.700833
407,407,13802.877727
408,408,14093.033333
409,409,15549.595417


In [151]:
c2.to_csv('F2.csv', index=False)
s2 = pd.read_csv('F2.csv')
s2

,Id,price
0,0,16961.997380
1,1,16644.733417
2,2,23029.462500
3,3,14841.529000
4,4,8503.470833
...,...,...
406,406,28386.700833
407,407,13802.877727
408,408,14093.033333
409,409,15549.595417
